In [1]:
import pymysql

db = pymysql.connect(host='162.105.146.37',user='stu1900011117', password='stu1900011117', port=53306,db="stu1900011117")
cursor = db.cursor()

In [2]:
cursor.execute("show databases;")
print(cursor.fetchall())
cursor.execute("show tables;")
print(cursor.fetchall())

(('information_schema',), ('stu1900011117',))
()


**步骤一：罗列业务需求**\
1.微信两个不同用户之间可以进行点对点通信，即A发送一条信息，B收到该条信息\
2.微信两个不同用户之间可以相互认识，成为朋友关系\
3.微信用户可以创建群，并且指定一个用户为该群群主\
4.微信用户可以由引介人引介后加入群\
5.微信用户可以在所在群中发送帖子，该帖子所在群中所有成员均可见\
6.微信用户可以创建一条朋友圈\
7.微信用户可以给朋友的朋友圈点赞，点赞仅共同好友可见，即B对A的朋友圈点赞，只有同时为B和A的朋友可见B的点赞\
8.微信用户可以对朋友的朋友圈进行评论，评论仅共同好友可见，规则同7

**步骤二：ER设计**\
图片见PDF版本\
![ER](D:\jupyter_notebook_code\database assignment\practice1-er.png)

**步骤三：数据库设计**\
对微信中的5个对象，分别建立属于每个对象的属性集合，具体属性描述如下：
- 用户（用户ID，昵称，性别，电话，生日，地区）
- 短信（短信ID，内容，时间）
- 群（群ID，群名称，群类型，创建时间，群主ID）
- 帖子（帖子ID，内容，时间）
- 朋友圈（朋友圈ID，内容，时间）

对象之间的关系：
- 一个用户可以和其他多个用户认识
- 一个用户可以发送多条短信，也可以接收多条短信
- 一个用户可以创建多个群，也可以加入多个群
- 一个用户可以发送多个帖子
- 一个群可以有多个帖子
- 一个用户可以发送多条朋友圈，也可以点赞、评论多条朋友圈

说明：
- 一个用户可以发送多条短信，也可以接收多条短信，并且每条短信具有唯一的发送用户和接收用户（群发视为一个用户分别发送给多个用户相同内容的短信，但每条短信ID不同），所以短信引用用户ID作为外键
- 一个用户可以发送多条帖子，并且每条帖子具有唯一的群ID，所以帖子引用用户ID和群ID作为外键
- 一个用户可以发送多条朋友圈，所以朋友圈引用用户ID作为外键
- 两个用户之间可以相互认识，所以生成关系表——认识表。认识表引用两个不同的用户ID作为外键，并且添加用户认识的时间和方式属性
- 一个用户可以加入多个群，一个群可以包括多个用户，所以生成关系表——群成员表。群成员表引用群ID、用户ID作为外键，并且添加引介人和引介时间属性
- 一个用户可以对多个朋友圈进行点赞和评论，所以分别生成关系表——点赞关系表、评论关系表。两个关系表均引用用户ID和朋友圈ID作为外键，评论关系表还添加时间和内容属性。使用视图控制点赞和评论的可见性

对象及属性对应的编号为：
- user(u_id, u_nickname, u_gender, u_area, u_phone, u_birthday)
- message(mes_id, mes_time, mes_content, mes_send_uid, mes_receive_uid)
- chat(c_id, c_name, c_type, c_create_time, c_owner_uid)
- post(p_id, p_time, p_content, p_send_uid, p_gid)
- moment(mom_id, mom_content, mom_time, mom_uid)
- acquaintance(a_id, a_uid1, a_uid2, a_time)
- groupMember(gm_id, gm_gid, gm_time, gm_intro_uid, gm_uid)
- likes(l_id, l_mid, l_uid)
- comment(com_id, com_mid, com_uid, com_time, com_content)

In [3]:
# 创建user表
sql = 'CREATE TABLE user (u_id INT PRIMARY KEY, u_nickname VARCHAR(50) NOT NULL, u_gender ENUM("F", "M"), u_area VARCHAR(20), u_phone CHAR(11), u_birthday date)'
cursor.execute(sql)

0

In [4]:
# 创建message表
sql = 'CREATE TABLE message (mes_id INT PRIMARY KEY, mes_time datetime NOT NULL, mes_content VARCHAR(255) NOT NULL, mes_send_uid INT NOT NULL, mes_receive_uid INT NOT NULL)'
cursor.execute(sql)

0

In [5]:
# 创建chat表
sql = 'CREATE TABLE chat (c_id INT PRIMARY KEY, c_name VARCHAR(20) NOT NULL, c_type VARCHAR(20), c_create_time datetime, c_owner_uid INT)'
cursor.execute(sql)

0

In [6]:
# 创建post表
sql = 'CREATE TABLE post (p_id INT PRIMARY KEY, p_time datetime, p_content VARCHAR(255) NOT NULL, p_send_uid INT, p_gid INT)'
cursor.execute(sql)

0

In [7]:
# 创建moment表
sql = 'CREATE TABLE moment (mom_id INT PRIMARY KEY, mom_content VARCHAR(255) NOT NULL, mom_time datetime NOT NULL, mom_uid INT)'
cursor.execute(sql)

0

In [8]:
# 创建acquaintance表
sql = 'CREATE TABLE acquaintance (a_id INT PRIMARY KEY, a_uid1 INT, a_uid2 INT, a_time datetime NOT NULL)'
cursor.execute(sql)

0

In [9]:
# 创建groupMember表
sql = 'CREATE TABLE groupMember (gm_id INT PRIMARY KEY, gm_gid INT, gm_time datetime NOT NULL, gm_intro_uid INT, gm_uid INT)'
cursor.execute(sql)

0

In [10]:
# 创建likes表
sql = 'CREATE TABLE likes (l_id INT PRIMARY KEY, l_mid INT, l_uid INT)'
cursor.execute(sql)

0

In [11]:
# 创建comment表
sql = 'CREATE TABLE comment (com_id INT PRIMARY KEY, com_mid INT, com_uid INT, com_time datetime NOT NULL, com_content VARCHAR(255) NOT NULL)'
cursor.execute(sql)

0

In [12]:
# 添加约束

# 对短信添加外键约束
sql = 'ALTER TABLE message ADD CONSTRAINT fk_mes_send FOREIGN KEY (mes_send_uid) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)
sql = 'ALTER TABLE message ADD CONSTRAINT fk_mes_receive FOREIGN KEY (mes_receive_uid) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)

# 对帖子添加外键约束
sql = 'ALTER TABLE post ADD CONSTRAINT fk_p_send FOREIGN KEY (p_send_uid) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)
sql = 'ALTER TABLE post ADD CONSTRAINT fk_p_chat FOREIGN KEY (p_gid) REFERENCES chat(c_id) ON DELETE CASCADE'
cursor.execute(sql)

# 对朋友圈添加外键约束
sql = 'ALTER TABLE moment ADD CONSTRAINT fk_mom_uid FOREIGN KEY (mom_uid) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)

# 对认识表添加外键约束
sql = 'ALTER TABLE acquaintance ADD CONSTRAINT fk_a_1 FOREIGN KEY (a_uid1) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)
sql = 'ALTER TABLE acquaintance ADD CONSTRAINT fk_a_2 FOREIGN KEY (a_uid2) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)

# 对群成员表添加外键约束
sql = 'ALTER TABLE groupMember ADD CONSTRAINT fk_gm_gid FOREIGN KEY (gm_gid) REFERENCES chat(c_id) ON DELETE CASCADE'
cursor.execute(sql)
sql = 'ALTER TABLE groupMember ADD CONSTRAINT fk_gm_intro FOREIGN KEY (gm_intro_uid) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)
sql = 'ALTER TABLE groupMember ADD CONSTRAINT fk_gm_uid FOREIGN KEY (gm_uid) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)

# 对点赞表添加外键约束
sql = 'ALTER TABLE likes ADD CONSTRAINT fk_l_mid FOREIGN KEY (l_mid) REFERENCES moment(mom_id) ON DELETE CASCADE'
cursor.execute(sql)
sql = 'ALTER TABLE likes ADD CONSTRAINT fk_l_uid FOREIGN KEY (l_uid) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)

# 对评论表添加外键约束
sql = 'ALTER TABLE comment ADD CONSTRAINT fk_com_mid FOREIGN KEY (com_mid) REFERENCES moment(mom_id) ON DELETE CASCADE'
cursor.execute(sql)
sql = 'ALTER TABLE comment ADD CONSTRAINT fk_com_uid FOREIGN KEY (com_uid) REFERENCES user(u_id) ON DELETE CASCADE'
cursor.execute(sql)

# 在认识关系中，两个用户不能为同一个用户，添加该约束
sql = 'ALTER TABLE acquaintance ADD CONSTRAINT a_not_equal CHECK (a_uid1 != a_uid2)'
cursor.execute(sql)

# 在短信中，发送用户和接收用户不能为同一个用户，添加该约束
sql = 'ALTER TABLE message ADD CONSTRAINT mes_not_equal CHECK (mes_send_uid != mes_receive_uid)'
cursor.execute(sql)

0

In [13]:
# 生成一些样例数据
# user
sql = 'INSERT INTO user(u_id, u_nickname, u_gender, u_area, u_phone, u_birthday) values(%s, %s, %s, %s, %s, %s)'
cursor.execute(sql, (1, 'wang1', 'M', 'beijing', '18800000001', '20000101'))
cursor.execute(sql, (2, 'wang2', 'M', 'beijing', '18800000002', '20000102'))
cursor.execute(sql, (3, 'wang3', 'M', 'shanghai', '18800000003', '20000103'))
cursor.execute(sql, (4, 'wang4', 'F', 'beijing', '18800000004', '20000104'))
cursor.execute(sql, (5, 'wang5', 'F', 'shanghai', '18800000005', '20000105'))
cursor.execute(sql, (6, 'wang6', 'F', 'shanghai', '18800000006', '20000106'))
cursor.execute(sql, (7, 'yu1', 'M', 'beijing', '18800000007', '20000107'))
cursor.execute(sql, (8, 'yu2', 'M', 'hebei', '18800000008', '20000108'))
cursor.execute(sql, (9, 'yu3', 'M', 'beijing', '18800000009', '20000109'))
cursor.execute(sql, (10, 'yu4', 'F', 'hebei', '18800000010', '20000110'))
cursor.execute(sql, (11, 'yu5', 'F', 'beijing', '18800000011', '20000201'))
cursor.execute(sql, (12, 'yu6', 'F', 'hebei', '18800000012', '20000301'))
cursor.execute(sql, (13, 'lin1', 'F', 'beijing', '18800000013', '20000401'))
cursor.execute(sql, (14, 'lin2', 'F', 'xinjiang', '18800000014', '20000501'))
cursor.execute(sql, (15, 'lin3', 'F', 'xinjiang', '18800000015', '20000601'))
cursor.execute(sql, (16, 'lin4', 'M', 'xinjiang', '18800000016', '20000701'))
cursor.execute(sql, (17, 'lin5', 'M', 'beijing', '18800000017', '20000801'))
cursor.execute(sql, (18, 'lin6', 'M', 'beijing', '18800000018', '20000901'))
cursor.execute(sql, (19, 'wu1', 'F', 'beijing', '18800000019', '20001001'))
cursor.execute(sql, (20, 'wu2', 'F', 'shanghai', '18800000020', '20000101'))
cursor.execute(sql, (21, 'wu3', 'F', 'beijing', '18800000021', '20000102'))
cursor.execute(sql, (22, 'wu4', 'M', 'shanghai', '18800000022', '20000103'))
cursor.execute(sql, (23, 'wu5', 'M', 'shanghai', '18800000023', '20000104'))
cursor.execute(sql, (24, 'wu6', 'M', 'shanghai', '18800000024', '20000105'))
cursor.execute(sql, (25, 'chen1', 'M', 'beijing', '18800000025', '20000106'))
cursor.execute(sql, (26, 'chen2', 'F', 'hebei', '18800000026', '20000107'))
cursor.execute(sql, (27, 'chen3', 'M', 'beijing', '18800000027', '20000108'))
cursor.execute(sql, (28, 'chen4', 'F', 'xinjiang', '18800000028', '20000109'))
cursor.execute(sql, (29, 'chen5', 'M', 'guangdong', '18800000029', '20000110'))
cursor.execute(sql, (30, 'chen6', 'F', 'henan', '18800000030', '20000101'))

# message
sql =  'INSERT INTO message(mes_id, mes_time, mes_content, mes_send_uid, mes_receive_uid) values(%s, %s, %s, %s, %s)'
cursor.execute(sql, (1, '20220505224400', 'a', '1', '2'))
cursor.execute(sql, (2, '20220505224401', 'a', '1', '3'))
cursor.execute(sql, (3, '20220505224402', 'aa', '1', '4'))
cursor.execute(sql, (4, '20220505224403', 'aaa', '1', '5'))
cursor.execute(sql, (5, '20220505224404', 'ab', '1', '6'))
cursor.execute(sql, (6, '20220505224405', 'abb', '1', '7'))
cursor.execute(sql, (7, '20220505224406', 'abbb', '2', '3'))
cursor.execute(sql, (8, '20220505224407', 'a', '2', '4'))
cursor.execute(sql, (9, '20220505224408', 'a', '2', '5'))
cursor.execute(sql, (10, '20220505224409', 'ab', '2', '6'))
cursor.execute(sql, (11, '20220505224400', 'abb', '1', '12'))
cursor.execute(sql, (12, '20220505224401', 'ac', '1', '13'))
cursor.execute(sql, (13, '20220504224400', 'ad', '3', '4'))
cursor.execute(sql, (14, '20220503224400', 'ae', '4', '5'))
cursor.execute(sql, (15, '20220502224400', 'af', '5', '6'))
cursor.execute(sql, (16, '20220501224400', 'ag', '2', '1'))
cursor.execute(sql, (17, '20220505224401', 'ah', '2', '1'))
cursor.execute(sql, (18, '20220505224402', 'ai', '2', '1'))
cursor.execute(sql, (19, '20220505224403', 'aj', '2', '1'))
cursor.execute(sql, (20, '20220505224404', 'ak', '2', '1'))

# chat
sql =  'INSERT INTO chat(c_id, c_name, c_type, c_create_time, c_owner_uid) values(%s, %s, %s, %s, %s)'
cursor.execute(sql, (1, 'a', 'game', '20220505224900', '1'))
cursor.execute(sql, (2, 'b', 'game', '20220405224900', '2'))
cursor.execute(sql, (3, 'c', 'game', '20220305224900', '3'))
cursor.execute(sql, (4, 'd', 'learn', '20220205224900', '1'))
cursor.execute(sql, (5, 'e', 'learn', '20220505224900', '5'))
cursor.execute(sql, (6, 'f', 'sport', '20220105224900', '6'))

# post
sql = 'INSERT INTO post(p_id, p_time, p_content, p_send_uid, p_gid) values(%s, %s, %s, %s, %s)'
cursor.execute(sql, (1, '20220505224900', 'a', '1', '1'))
cursor.execute(sql, (2, '20220505224901', 'b', '2', '1'))
cursor.execute(sql, (3, '20220505224902', 'c', '3', '1'))
cursor.execute(sql, (4, '20220505224903', 'd', '4', '1'))
cursor.execute(sql, (5, '20220505224904', 'a', '1', '2'))
cursor.execute(sql, (6, '20220505224905', 'b', '2', '2'))
cursor.execute(sql, (7, '20220505224906', 'c', '3', '2'))
cursor.execute(sql, (8, '20220505224907', 'd', '4', '2'))
cursor.execute(sql, (9, '20220505224900', 'a', '1', '3'))
cursor.execute(sql, (10, '20220505224900', 'a', '1', '4'))

# moment
sql = 'INSERT INTO moment(mom_id, mom_content, mom_time, mom_uid) values(%s, %s, %s, %s)'
cursor.execute(sql, (1, 'a', '20220505224900', '1'))
cursor.execute(sql, (2, 'b', '20220505224901', '1'))
cursor.execute(sql, (3, 'e', '20220505224902', '2'))
cursor.execute(sql, (4, 'c', '20220505224902', '3'))
cursor.execute(sql, (5, 'd', '20220505224903', '4'))

# acquaintance
sql = 'INSERT INTO acquaintance(a_id, a_uid1, a_uid2, a_time) values(%s, %s, %s, %s)'
cursor.execute(sql, (1, '1', '2', '20220505224900'))
cursor.execute(sql, (2, '1', '3', '20220505224901'))
cursor.execute(sql, (3, '1', '4', '20220505224902'))
cursor.execute(sql, (4, '1', '5', '20220505224902'))
cursor.execute(sql, (5, '2', '3', '20220505224903'))
cursor.execute(sql, (6, '2', '4', '20220505224900'))
cursor.execute(sql, (7, '2', '5', '20220505224901'))
cursor.execute(sql, (8, '5', '6', '20220505224902'))
cursor.execute(sql, (9, '3', '4', '20220505224902'))
cursor.execute(sql, (10, '6', '9', '20220505224903'))
cursor.execute(sql, (11, '2', '1', '20220505224900'))
cursor.execute(sql, (12, '3', '1', '20220505224901'))
cursor.execute(sql, (13, '4', '1', '20220505224902'))
cursor.execute(sql, (14, '5', '1', '20220505224902'))
cursor.execute(sql, (15, '3', '2', '20220505224903'))
cursor.execute(sql, (16, '4', '2', '20220505224900'))
cursor.execute(sql, (17, '5', '2', '20220505224901'))
cursor.execute(sql, (18, '6', '5', '20220505224902'))
cursor.execute(sql, (19, '4', '3', '20220505224902'))
cursor.execute(sql, (20, '9', '6', '20220505224903'))

# groupMember
sql = 'INSERT INTO groupMember(gm_id, gm_gid, gm_time, gm_intro_uid, gm_uid) values(%s, %s, %s, %s, %s)'
cursor.execute(sql, (1, '1', '20220505224900', '1', '1'))
cursor.execute(sql, (2, '1', '20220505224901', '1', '2'))
cursor.execute(sql, (3, '1', '20220505224902', '2', '3'))
cursor.execute(sql, (4, '1', '20220505224902', '2', '4'))
cursor.execute(sql, (5, '2', '20220505224903', '2', '2'))
cursor.execute(sql, (6, '2', '20220505224900', '2', '1'))
cursor.execute(sql, (7, '2', '20220505224901', '1', '3'))
cursor.execute(sql, (8, '2', '20220505224902', '1', '4'))
cursor.execute(sql, (9, '3', '20220505224902', '3', '3'))
cursor.execute(sql, (10, '3', '20220505224903', '3', '1'))
cursor.execute(sql, (11, '4', '20220505224902', '1', '1'))
cursor.execute(sql, (12, '5', '20220505224902', '5', '5'))
cursor.execute(sql, (13, '6', '20220505224903', '6', '6'))

# likes
sql = 'INSERT INTO likes(l_id, l_mid, l_uid) values(%s, %s, %s)'
cursor.execute(sql, (1, '1', '1'))
cursor.execute(sql, (2, '1', '2'))
cursor.execute(sql, (3, '1', '3'))
cursor.execute(sql, (4, '2', '4'))
cursor.execute(sql, (5, '2', '5'))
cursor.execute(sql, (6, '1', '4'))
cursor.execute(sql, (7, '1', '5'))

# comment
sql = 'INSERT INTO comment(com_id, com_mid, com_uid, com_time, com_content) values(%s, %s, %s, %s, %s)'
cursor.execute(sql, (1, '1', '2', '20220505224903', 'a'))
cursor.execute(sql, (2, '1', '3', '20220505224904', 'b'))
cursor.execute(sql, (3, '2', '3', '20220505224903', 'c'))

1

**步骤四：数据库操作**\
\
*查询操作*

In [14]:
# 1.单表查询

# 查询群信息表
sql = 'SELECT c_id, c_owner_uid, c_name FROM chat'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    print('c_id, c_owner_uid, c_name')
    for row in results:
        print(row)
except Exception as e:
    print(e)

c_id, c_owner_uid, c_name
(1, 1, 'a')
(2, 2, 'b')
(3, 3, 'c')
(4, 1, 'd')
(5, 5, 'e')
(6, 6, 'f')


In [15]:
# 2.条件查询

# 查询来自北京的用户信息
sql = 'SELECT u_id, u_nickname, u_phone FROM user WHERE u_area = "beijing" '
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    print('u_id, u_nickname, u_phone')
    for row in results:
        print(row)
except Exception as e:
    print(e)

u_id, u_nickname, u_phone
(1, 'wang1', '18800000001')
(2, 'wang2', '18800000002')
(4, 'wang4', '18800000004')
(7, 'yu1', '18800000007')
(9, 'yu3', '18800000009')
(11, 'yu5', '18800000011')
(13, 'lin1', '18800000013')
(17, 'lin5', '18800000017')
(18, 'lin6', '18800000018')
(19, 'wu1', '18800000019')
(21, 'wu3', '18800000021')
(25, 'chen1', '18800000025')
(27, 'chen3', '18800000027')


In [16]:
# 3.聚合查询

# 查询来自不同省份的用户个数，并按照个数从少到多排序
sql = 'SELECT u_area, count(*) FROM user GROUP BY u_area ORDER BY count(*)'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print(e)

('guangdong', 1)
('henan', 1)
('hebei', 4)
('xinjiang', 4)
('shanghai', 7)
('beijing', 13)


In [17]:
# 4.连接查询

# 查询加入群聊的用户的ID，电话和地区
sql = 'SELECT u_id, u_phone, u_area FROM user WHERE EXISTS (SELECT * FROM groupMember g WHERE g.gm_uid = user.u_id)'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    print('u_id, u_phone, u_area')
    for row in results:
        print(row)
except Exception as e:
    print(e)

u_id, u_phone, u_area
(1, '18800000001', 'beijing')
(2, '18800000002', 'beijing')
(3, '18800000003', 'shanghai')
(4, '18800000004', 'beijing')
(5, '18800000005', 'shanghai')
(6, '18800000006', 'shanghai')


In [18]:
# 5.子查询

# 查询给1号用户的好友发送过信息的用户ID
sql = 'SELECT DISTINCT mes_send_uid FROM message m WHERE m.mes_send_uid != "1" AND m.mes_receive_uid IN (SELECT a_uid2 FROM acquaintance a WHERE a.a_uid1 = "1")'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    print('u_id')
    for row in results:
        print(row)
except Exception as e:
    print(e)

u_id
(2,)
(3,)
(4,)


In [19]:
# 6.WITH AS

# 查询帖子信息表并按时间倒序排序
sql = 'WITH temp AS (SELECT * FROM post ORDER BY p_time DESC) SELECT * FROM temp'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    print('u_id')
    for row in results:
        print(row)
except Exception as e:
    print(e)

u_id
(8, datetime.datetime(2022, 5, 5, 22, 49, 7), 'd', 4, 2)
(7, datetime.datetime(2022, 5, 5, 22, 49, 6), 'c', 3, 2)
(6, datetime.datetime(2022, 5, 5, 22, 49, 5), 'b', 2, 2)
(5, datetime.datetime(2022, 5, 5, 22, 49, 4), 'a', 1, 2)
(4, datetime.datetime(2022, 5, 5, 22, 49, 3), 'd', 4, 1)
(3, datetime.datetime(2022, 5, 5, 22, 49, 2), 'c', 3, 1)
(2, datetime.datetime(2022, 5, 5, 22, 49, 1), 'b', 2, 1)
(1, datetime.datetime(2022, 5, 5, 22, 49), 'a', 1, 1)
(9, datetime.datetime(2022, 5, 5, 22, 49), 'a', 1, 3)
(10, datetime.datetime(2022, 5, 5, 22, 49), 'a', 1, 4)


*视图（用于限制朋友圈的可见性）*

In [20]:
# 查看用户3可见的朋友圈点赞

# （1）查看全部朋友圈点赞信息
sql = 'SELECT * FROM likes'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print(e)

(1, 1, 1)
(2, 1, 2)
(3, 1, 3)
(4, 2, 4)
(5, 2, 5)
(6, 1, 4)
(7, 1, 5)


In [21]:
# （2）创建用户3可见的朋友圈点赞视图
sql = 'CREATE VIEW likes_3 as SELECT DISTINCT l_id, l_mid, l_uid FROM likes, acquaintance a, moment m WHERE (a.a_uid1 = likes.l_uid AND a.a_uid2 = "3" AND m.mom_id = likes.l_mid AND m.mom_uid IN (SELECT a_uid2 FROM acquaintance a WHERE a.a_uid1 = "3")) OR (l_uid = "3") OR (m.mom_id = likes.l_mid AND m.mom_uid = "3")'
cursor.execute(sql)
sql = 'SELECT * FROM likes_3'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print(e)

(3, 1, 3)
(1, 1, 1)
(2, 1, 2)
(4, 2, 4)
(6, 1, 4)


In [22]:
# （3）在视图的基础上，查询用户3所能看见的1号朋友圈的点赞信息
sql = 'SELECT * FROM likes_3 WHERE l_mid = "1"'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print(e)

(3, 1, 3)
(1, 1, 1)
(2, 1, 2)
(6, 1, 4)


In [23]:
# （4）删除视图
sql = 'DROP VIEW likes_3'
cursor.execute(sql)

0

*数据的修改和增删*

In [24]:
# 修改数据

# 将所有以用户1为群主的群改为以用户2作为群主（需要先检查用户2是否在群中，若不在需要添加用户2至该群中）
# （1）查看以用户1为群主的群
sql = 'SELECT * FROM chat WHERE c_owner_uid = "1"'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print(e)

(1, 'a', 'game', datetime.datetime(2022, 5, 5, 22, 49), 1)
(4, 'd', 'learn', datetime.datetime(2022, 2, 5, 22, 49), 1)


In [25]:
# （2）若用户2不在群中，则添加用户2至该群中
sql = 'INSERT INTO groupMember(gm_id, gm_gid, gm_time, gm_intro_uid, gm_uid) SELECT "14", "1", "20220505224903", "1", "2" FROM DUAL WHERE NOT EXISTS (SELECT * FROM groupMember WHERE gm_uid = "2" AND gm_gid = "1")'
cursor.execute(sql)
sql = 'INSERT INTO groupMember(gm_id, gm_gid, gm_time, gm_intro_uid, gm_uid) SELECT "15", "4", "20220505224903", "1", "2" FROM DUAL WHERE NOT EXISTS (SELECT * FROM groupMember WHERE gm_uid = "2" AND gm_gid = "4")'
cursor.execute(sql)

1

In [26]:
# （3）更新数据，并查看结果
sql = 'UPDATE chat SET c_owner_uid = "2" WHERE c_owner_uid = "1"'
cursor.execute(sql)
sql = 'SELECT * FROM chat'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print(e)

(1, 'a', 'game', datetime.datetime(2022, 5, 5, 22, 49), 2)
(2, 'b', 'game', datetime.datetime(2022, 4, 5, 22, 49), 2)
(3, 'c', 'game', datetime.datetime(2022, 3, 5, 22, 49), 3)
(4, 'd', 'learn', datetime.datetime(2022, 2, 5, 22, 49), 2)
(5, 'e', 'learn', datetime.datetime(2022, 5, 5, 22, 49), 5)
(6, 'f', 'sport', datetime.datetime(2022, 1, 5, 22, 49), 6)


In [27]:
# 删除数据

# 删除2发送给1的所有短信记录
# （1）查看删除前用户2的所有发送短信信息
sql = 'SELECT * FROM message WHERE mes_send_uid = "2"'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print(e)

(7, datetime.datetime(2022, 5, 5, 22, 44, 6), 'abbb', 2, 3)
(8, datetime.datetime(2022, 5, 5, 22, 44, 7), 'a', 2, 4)
(9, datetime.datetime(2022, 5, 5, 22, 44, 8), 'a', 2, 5)
(10, datetime.datetime(2022, 5, 5, 22, 44, 9), 'ab', 2, 6)
(16, datetime.datetime(2022, 5, 1, 22, 44), 'ag', 2, 1)
(17, datetime.datetime(2022, 5, 5, 22, 44, 1), 'ah', 2, 1)
(18, datetime.datetime(2022, 5, 5, 22, 44, 2), 'ai', 2, 1)
(19, datetime.datetime(2022, 5, 5, 22, 44, 3), 'aj', 2, 1)
(20, datetime.datetime(2022, 5, 5, 22, 44, 4), 'ak', 2, 1)


In [28]:
# （2）删除并查看结果
sql = 'DELETE FROM message WHERE mes_send_uid = "2" AND mes_receive_uid = "1"'
cursor.execute(sql)
sql = 'SELECT * FROM message WHERE mes_send_uid = "2"'
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except Exception as e:
    print(e)

(7, datetime.datetime(2022, 5, 5, 22, 44, 6), 'abbb', 2, 3)
(8, datetime.datetime(2022, 5, 5, 22, 44, 7), 'a', 2, 4)
(9, datetime.datetime(2022, 5, 5, 22, 44, 8), 'a', 2, 5)
(10, datetime.datetime(2022, 5, 5, 22, 44, 9), 'ab', 2, 6)
